In [2]:
import pandas as pd
import numpy as np

In [3]:
space_units_df = pd.read_csv('../Data/space_units_df.csv', index_col='Name')
ground_units_df= pd.read_csv('../Data/ground_units_df.csv', index_col='Name')

In [5]:
# Set Ships

faction_A_units = {'War Sun': 2}
faction_B_units = {'Dreadnought': 4}

In [6]:
# Find Combat Values

def get_ship_stats(faction_units):
    faction_ship_stats = []
    for unit in faction_units:
        number_units = faction_units[unit]
        for i in range(number_units):
            name = unit
            unit_combat_value = int(space_units_df.loc[unit]['Combat_Value'])
            has_sustain_damage = space_units_df.loc[unit]['Has_Sustain_Damage']
            shots = space_units_df.loc[unit]['Shots']
            ship_stats = {'Name': name, 'Unit_Combat_Value': unit_combat_value, 'Shots': shots, 'Has_Sustain_Damage': has_sustain_damage}
            faction_ship_stats.append(ship_stats)
    return faction_ship_stats

faction_A_ships = get_ship_stats(faction_A_units)
faction_B_ships = get_ship_stats(faction_B_units)

In [7]:
# Determine Hits

def get_ship_hits(faction_ships):
    hits = 0
    for ship in faction_ships:
        for i in range(ship['Shots']):
            roll = np.random.randint(1, 11)
            if roll >= ship['Unit_Combat_Value']:
                hits += 1
    return hits

faction_A_hits = get_ship_hits(faction_A_ships)
faction_B_hits = get_ship_hits(faction_B_ships)

In [8]:
# Assign Hits

def assign_hits(hits, faction_to_assign):
    extra_hits = 0
    for i in range(hits):
        try:
            if faction_to_assign[0]['Has_Sustain_Damage']:
                faction_to_assign[0]['Has_Sustain_Damage'] = False
            else:
                faction_to_assign.remove(faction_to_assign[0])
        except IndexError:
            extra_hits += 1
    return extra_hits

extra_hits_A = assign_hits(faction_A_hits, faction_B_ships)
extra_hits_B = assign_hits(faction_B_hits, faction_A_ships)

In [9]:
faction_A_ships

[{'Name': 'War Sun',
  'Unit_Combat_Value': 3,
  'Shots': np.int64(3),
  'Has_Sustain_Damage': np.True_}]

In [10]:
faction_B_ships

[{'Name': 'Dreadnought',
  'Unit_Combat_Value': 5,
  'Shots': np.int64(1),
  'Has_Sustain_Damage': False},
 {'Name': 'Dreadnought',
  'Unit_Combat_Value': 5,
  'Shots': np.int64(1),
  'Has_Sustain_Damage': np.True_}]

In [11]:
# MAIN


faction_A_wins, faction_B_wins, draws = 0, 0, 0
round_counts = []

for i in range(0, 1000):

    faction_A_units = {'Arc Secundus': 1, 'Carrier': 2, 'Fighter': 8}
    faction_B_units = {'War Sun': 1, 'Carrier': 2, 'Fighter': 8}

    # Find Combat Values
    faction_A_ships = get_ship_stats(faction_A_units)
    faction_B_ships = get_ship_stats(faction_B_units)

    round_count = 0
    while faction_A_ships and faction_B_ships:
        # Determine Hits
        faction_A_hits = get_ship_hits(faction_A_ships)
        faction_B_hits = get_ship_hits(faction_B_ships)

        # Assign Hits
        assign_hits(faction_A_hits, faction_B_ships)
        assign_hits(faction_B_hits, faction_A_ships)

        #print(faction_A_ships)
        #print(faction_B_ships)
        round_count += 1
    
    round_counts.append(round_count)

    if faction_A_ships:
        faction_A_wins += 1
    elif faction_B_ships:
        faction_B_wins += 1
    else:
        draws += 1

average_num_rounds = sum(round_counts) / len(round_counts)
print(f'Average Number of Rounds: {average_num_rounds}')

total_games = faction_A_wins + faction_B_wins + draws
print(f'Number of Games Ran: {total_games}')

faction_A_win_percentage = (faction_A_wins / total_games) * 100
faction_B_win_percentage = (faction_B_wins /total_games) * 100
draw_percentage = draws / (total_games) * 100

df = pd.DataFrame([{'Faction A wins': faction_A_win_percentage, 'Faction B wins': faction_B_win_percentage, 'Draw': draw_percentage}], 
                  index=['Percent'])
df.round(1)

Average Number of Rounds: 6.555
Number of Games Ran: 1000


,Faction A wins,Faction B wins,Draw
Percent,31.9,67.7,0.4
